Lab 4 

In [4]:
# Build  simple UDP server

import socket # imports built in socket module

server_socket = socket.socket(socket.AF_INET , socket.SOCK_DGRAM) # creates socket object with specific ipv4, UDP socket
server_socket.bind(('localhost', 65433)) # binds socker to specifc port number

print ("UDP server is ready to recieve API data...")

while True:
    data, client_address = server_socket.recvfrom(2048) # returns data and address of sender
    print(f"recieved data from {client_address}: {data.decode()}") # prints sender ddress and converts to readable string
    

UDP server is ready to recieve API data...
recieved data from ('127.0.0.1', 62337): Hello, udp server!


KeyboardInterrupt: 

In [ ]:
# Client code


import socket

client_socket = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)

server_address = ('localhost', 65433)
message = b"Hello, udp server!"

client_socket.sendto(message, server_address)

client_socket.close()

In [5]:
# Exercise 1 

# server side

import socket

server_socket = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)
server_socket.bind(('localhost', 65433))

print("UDP Chat Server is running...")

while True:
    data, client_address = server_socket.recvfrom(2048)
    message = data.decode()
    print(f"Client ({client_address}): {message}")

    response = input("You (Server): ")
    server_socket.sendto(response.encode(), client_address)

server_socket.close()

UDP Chat Server is running...
Client (('127.0.0.1', 51284)): hi
Client (('127.0.0.1', 51284)): hi


KeyboardInterrupt: 

In [ ]:
# client side

import socket

client_socket = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)
server_address = ('localhost', 65433)

print("Connected to UDP Chat Server")

while True:
    message = input("You (Client): ")
    client_socket.sendto(message.encode(), server_address)
    
    response, _ = client_socket.recvfrom(2048)
    print(f"Server: {response.decode()}")

client_socket.close()


In [ ]:
# Exercise 2 

# server side

import socket

server_socket = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)
server_socket.bind(('localhost', 65433))

clients = {}

print("UDP Chat Server is running...")

while True:
    data, client_address = server_socket.recvfrom(2048)
    message = data.decode()

    if client_address not in clients:
        clients[client_address] = f"User{len(clients) + 1}"
        print(f"New client: {clients[client_address]} ({client_address})")
    
    print(f"Client ({client_address}): {message}")

    response = input("You (Server): ")
    server_socket.sendto(response.encode(), client_address)

server_socket.close()

In [1]:
import socket
import json
import base64
from cryptography.fernet import Fernet

# Dictionary to store connected users: address: username
new_users = {}
# Dictionary to store user credentials: username: password
user_details = {
    "nicole": "shiki2014",
    "nic": "London34",
    "joanne": "Beth2005"
}

encryption_key = Fernet.generate_key()
cipher_suite = Fernet(encryption_key)

def encrypt_message(message):
    """Encrypt a message using Fernet symmetric encryption"""
    return cipher_suite.encrypt(message.encode())

def decrypt_message(encrypted_message):
    """Decrypt a message using Fernet symmetric encryption"""
    return cipher_suite.decrypt(encrypted_message).decode()

def is_authenticated(username, password):
    """Check if username and password match stored credentials"""
    return username in user_details and user_details[username] == password

def authentication(client_socket, client_address):
    """Handle the authentication process for a new client"""
    # Send authentication request
    auth_request = json.dumps({"type": "auth_request"})
    client_socket.sendto(auth_request.encode(), client_address)
    
    # Wait for credentials response (blocking)
    try:
        data, addr = client_socket.recvfrom(2048)
        auth_data = json.loads(data.decode())
        
        if auth_data["type"] == "auth_response":
            username = auth_data["username"]
            password = auth_data["password"]
            
            if is_authenticated(username, password):
                # Authentication successful
                new_users[client_address] = username
                response = json.dumps({"type": "auth_result", "successful": True, "key": encryption_key.decode()})
                client_socket.sendto(response.encode(), client_address)
                broadcast_message(f"SERVER: {username} has joined chat", client_address)
                print(f"User {username} authenticated from {client_address}")
                return True
            else:
                # Authentication failed
                response = json.dumps({"type": "auth_result", "success": False})
                client_socket.sendto(response.encode(), client_address)
                print(f"Failed from {client_address}")
                return False
    except Exception as e:
        print(f"Authentication error: {e}")
        return False

def broadcast_message(message, sender_address=None):
    """Send a message to all connected clients except the sender"""
    for address in new_users:
        if address != sender_address:
            encrypted_msg = encrypt_message(message)
            server_socket.sendto(
                json.dumps({
                    "type": "chat_message", 
                    "content": base64.b64encode(encrypted_msg).decode()
                }).encode(), 
                address
            )

# Create a UDP socket
server_socket = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)
server_socket.bind(('localhost', 65433))

print("UDP chat server is running on port 65433...")

while True:
    data, client_address = server_socket.recvfrom(2048)
    
    try:
        received_data = json.loads(data.decode())
        
        # Handle different types of messages
        if received_data["type"] == "connect":
            # New client connecting, initiate authentication
            authentication(server_socket, client_address)
        
        elif received_data["type"] == "chat_message":
            # Handle chat message from authenticated user
            if client_address in new_users:
                # Decrypt the message
                encrypted_content = base64.b64decode(received_data["content"])
                decrypted_message = decrypt_message(encrypted_content)
                
                # Format and broadcast the message
                sender_username = new_users[client_address]
                formatted_message = f"{sender_username}: {decrypted_message}"
                print(formatted_message)
                broadcast_message(formatted_message, client_address)
            else:
                # Unauthenticated user trying to send a message
                server_socket.sendto(json.dumps({"type": "error", "message": "Not authenticated"}).encode(), client_address)
        
        elif received_data["type"] == "disconnect":
            # Client disconnecting
            if client_address in new_users:
                username = new_users[client_address]
                broadcast_message(f"SERVER: {username} has left the chat", client_address)
                del new_users[client_address]
                print(f"User {username} disconnected from {client_address}")
    
    except Exception as e:
        print(f"Error with message: {e}")





UDP chat server is running on port 65433...


KeyboardInterrupt: 